In [1]:
import numpy as np
import tensorflow as tf
import h5py
import random

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 数据准备

In [2]:
#读取数据
f=h5py.File('D:/data/wind/data_4.h5','r')
data_x=f['x']
data_y=f['y']
data_y=np.expand_dims(data_y,axis=2)

#分配训练测试集
num=list(range(data_x.shape[0]))
num1=random.sample(num,20)
num2=set(num)-set(num1)
num2=list(num2)
num1.sort()


x_train=data_x[num2]
y_train=data_y[num2]

x_test=data_x[num1]
y_test=data_y[num1]
#生成批次
train_queue = tf.train.slice_input_producer([x_train,y_train],shuffle=None)
val_queue = tf.train.slice_input_producer([x_test,y_test],shuffle=None)
batch_xt,batch_yt=tf.train.shuffle_batch(train_queue,batch_size=16,capacity=500,min_after_dequeue=150)
batch_xv,batch_yv=tf.train.shuffle_batch(val_queue,batch_size=16,capacity=500,min_after_dequeue=150)

# 搭建模型

### 超参数设置

In [3]:
seq_length = batch_xt.shape[1]  # Time series will have the same past and future (to be predicted) lenght. 
batch_size = batch_xt.shape[0]  # Low value used for live demo purposes - 100 and 1000 would be possible too, crank that up!

output_dim = 1
input_dim = batch_xt.shape[-1]  # Output dimension (e.g.: multiple signals at once, tied in time)
hidden_dim = 24  # Count of hidden neurons in the recurrent units. 
layers_stacked_count = 3  # Number of stacked recurrent cells, on the neural depth axis. 

# Optmizer: 
learning_rate = 0.007  # Small lr helps not to diverge during training. 
nb_iters = 10000  # How many times we perform a training step (therefore how many times we show a batch). 
lr_decay = 0.92  # default: 0.9 . Simulated annealing.
momentum = 0.5  # default: 0.0 . Momentum technique in weights update
lambda_l2_reg = 0.003  # L2 regularization of weights - avoids overfitting

### 搭建s2s

In [4]:
with tf.variable_scope('encoder'):

    # Encoder: inputs
    enc_inp = tf.placeholder(tf.float32, shape=(None, 10,10), name="inp")

    # Decoder: expected outputs
    expected_sparse_output = tf.placeholder(tf.float32, shape=(None, 10,1), name="expected_sparse_output")
    pharse=tf.placeholder(tf.bool,name='if_train')
    
   

    # encoder cell 
    cells = []
    for i in range(layers_stacked_count):
        with tf.variable_scope('RNN_{}'.format(i)):
            cells.append(tf.contrib.rnn.GRUCell(hidden_dim))
            # cells.append(tf.nn.rnn_cell.BasicLSTMCell(...))
    cell = tf.contrib.rnn.MultiRNNCell(cells)
    
    
    enc_outputs, enc_memory = tf.nn.dynamic_rnn(cell,enc_inp,dtype=tf.float32)

with tf.variable_scope('decoder'):  
    decells=[]
    for i in range(layers_stacked_count):
        with tf.variable_scope('RNN_{}'.format(i)):
            decells.append(tf.contrib.rnn.GRUCell(hidden_dim))
            
    decell = tf.contrib.rnn.MultiRNNCell(decells)
    dec_state=enc_memory
    dec_inp=enc_outputs[:,-1,:]
    dec_outputs=[]
    #reshape expected_sparse_output
    w_h = tf.Variable(tf.random_normal([output_dim,hidden_dim]))
    b_h = tf.Variable(tf.random_normal([hidden_dim]))
    dec_inputs=[tf.matmul(expected_sparse_output[:,i,:],w_h)+b_h for i in range(seq_length)]
    
    for time_step in range(seq_length):
        if time_step>0:tf.get_variable_scope().reuse_variables()
        (dec_output,dec_state)=decell(dec_inp,dec_state)
        if pharse==True:
            dec_inp=dec_inputs[time_step]
        else:
            dec_inp=dec_output
        dec_outputs.append(dec_output)
    
    # reshape seq2seq输出 
    w_out = tf.Variable(tf.random_normal([hidden_dim, output_dim]))
    b_out = tf.Variable(tf.random_normal([output_dim]))
    
    # 最终输出
    output_scale_factor = tf.Variable(1.0, name="Output_ScaleFactor")
    
    reshaped_outputs = [output_scale_factor*(tf.matmul(i, w_out) + b_out) for i in dec_outputs]

### 搭建损失函数 

In [5]:
# Training loss and optimizer

with tf.variable_scope('Loss'):
    # L2 loss
    output_loss = 0
    for i in range(len(reshaped_outputs)):
        output_loss += tf.reduce_mean(tf.nn.l2_loss(reshaped_outputs[i] - expected_sparse_output[:,i,:]))
        
    # L2 regularization (to avoid overfitting and to have a  better generalization capacity)
    reg_loss = 0
    for tf_var in tf.trainable_variables():
        if not ("Bias" in tf_var.name or "Output_" in tf_var.name):
            reg_loss += tf.reduce_mean(tf.nn.l2_loss(tf_var))
            
    loss = output_loss + lambda_l2_reg * reg_loss
    tf.summary.scalar('loss',loss)

with tf.variable_scope('acc'):
    acc=0
    num=0
    for i in range(len(reshaped_outputs)):
        acc += tf.reduce_mean(tf.nn.l2_loss(reshaped_outputs[i] - expected_sparse_output[:,i,:]))/tf.reduce_mean(tf.nn.l2_loss( expected_sparse_output[:,i,:]-0))
        num=num+1
    acc=acc/num
    acc=1-acc
    tf.summary.scalar('acc',acc)

with tf.variable_scope('Optimizer'):
    optimizer = tf.train.RMSPropOptimizer(learning_rate, decay=lr_decay, momentum=momentum)
    train_op = optimizer.minimize(loss)
    merged_summary = tf.summary.merge_all()

### 训练模型

In [6]:
min_loss=10000
sess=tf.Session()
train_writer=tf.summary.FileWriter('D:/graph/wind-4-1/train/',sess.graph)
test_writer = tf.summary.FileWriter('D:/graph/wind-4-1/test/', sess.graph)
saver=tf.train.Saver(max_to_keep=3)

init_op = tf.global_variables_initializer()
sess.run(init_op)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess, coord)

for training_itr in range(10000):
    x1, y1 = sess.run([batch_xt,batch_yt])
    feed_dict1 = {enc_inp: x1}
    feed_dict1.update({expected_sparse_output: y1})
    feed_dict1.update({pharse:True})
    _, loss1,acc1,summaries1 = sess.run([train_op, loss,acc,merged_summary], feed_dict1)

    train_writer.add_summary(summaries1, training_itr)
    if training_itr %5==0:
#             saver.save(sess=sess, save_path='model/hand_landmark_v6.1_model/model.ckpt',global_step=(global_step + 1))
        mean_val_loss = 0

        x2,y2=sess.run([batch_xv,batch_yv])
        feed_dict2 = {enc_inp: x2}
        feed_dict2.update({expected_sparse_output: y2})
        feed_dict2.update({pharse:False})
        loss2,acc2,summaries2 = sess.run([loss,acc,merged_summary], feed_dict2)

        print('step: {}'.format(training_itr))
        print('train: loss: {} acc: {}  val: loss: {} acc: {}'.format(loss1,acc1,loss2,acc2))
        test_writer.add_summary(summaries2, training_itr)
        if loss1 < min_loss:
            min_loss=loss1
            saver.save(sess=sess, save_path='D:/model/wind-4-1/model.ckpt',global_step=(training_itr + 1))
sess.close()
coord.request_stop()
    






step: 0
train: loss: 4574.5302734375 acc: -0.5366077423095703  val: loss: 3006.13623046875 acc: 0.6289818286895752
step: 5
train: loss: 1145.770263671875 acc: 0.8569021224975586  val: loss: 1012.7208862304688 acc: 0.8355456590652466
step: 10
train: loss: 1494.511474609375 acc: 0.8006479740142822  val: loss: 1523.33837890625 acc: 0.6977113485336304
step: 15
train: loss: 886.3568115234375 acc: 0.8470785617828369  val: loss: 1098.5958251953125 acc: 0.7923693060874939
step: 20
train: loss: 655.4415893554688 acc: 0.8601730465888977  val: loss: 725.5482788085938 acc: 0.8896011710166931
step: 25
train: loss: 953.4411010742188 acc: 0.8364667296409607  val: loss: 758.8779907226562 acc: 0.9028199315071106
step: 30
train: loss: 1226.9429931640625 acc: 0.7341808080673218  val: loss: 1428.004150390625 acc: 0.7949180603027344
step: 35
train: loss: 436.9747314453125 acc: 0.9013840556144714  val: loss: 928.2267456054688 acc: 0.873285710811615
step: 40
train: loss: 1213.9058837890625 acc: 0.79722660779

step: 355
train: loss: 399.9646911621094 acc: 0.9098789095878601  val: loss: 1086.09814453125 acc: 0.8705436587333679
step: 360
train: loss: 829.163330078125 acc: 0.8933086395263672  val: loss: 1347.9613037109375 acc: 0.8198961019515991
step: 365
train: loss: 1108.79150390625 acc: 0.7231707572937012  val: loss: 1211.6453857421875 acc: 0.8331088423728943
step: 370
train: loss: 942.7859497070312 acc: 0.8076850175857544  val: loss: 2445.3876953125 acc: 0.6823176145553589
step: 375
train: loss: 752.0897827148438 acc: 0.8933994770050049  val: loss: 424.9921569824219 acc: 0.9280794262886047
step: 380
train: loss: 520.6289672851562 acc: 0.9164575934410095  val: loss: 1407.4329833984375 acc: 0.8149899244308472
step: 385
train: loss: 826.4861450195312 acc: 0.8457160592079163  val: loss: 900.789794921875 acc: 0.845691442489624
step: 390
train: loss: 787.5335083007812 acc: 0.883771538734436  val: loss: 2362.63525390625 acc: 0.7575620412826538
step: 395
train: loss: 685.496826171875 acc: 0.8780579

step: 705
train: loss: 963.4010009765625 acc: 0.8473922610282898  val: loss: 610.527099609375 acc: 0.9130348563194275
step: 710
train: loss: 471.2832946777344 acc: 0.9258319735527039  val: loss: 1353.1058349609375 acc: 0.8210476636886597
step: 715
train: loss: 921.9700317382812 acc: 0.8227491974830627  val: loss: 501.0044250488281 acc: 0.9166448712348938
step: 720
train: loss: 874.2536010742188 acc: 0.8066072463989258  val: loss: 726.2216796875 acc: 0.849925696849823
step: 725
train: loss: 798.969970703125 acc: 0.8434537649154663  val: loss: 2302.61669921875 acc: 0.7688347697257996
step: 730
train: loss: 712.1490478515625 acc: 0.8575664162635803  val: loss: 662.0975341796875 acc: 0.879362165927887
step: 735
train: loss: 766.4188842773438 acc: 0.8654465675354004  val: loss: 987.3771362304688 acc: 0.8126519322395325
step: 740
train: loss: 958.3605346679688 acc: 0.8164791464805603  val: loss: 1182.1121826171875 acc: 0.8681948184967041
step: 745
train: loss: 475.1675720214844 acc: 0.865992

step: 1065
train: loss: 461.7975769042969 acc: 0.907728374004364  val: loss: 1575.2274169921875 acc: 0.815043568611145
step: 1070
train: loss: 757.7072143554688 acc: 0.8366124033927917  val: loss: 795.0546264648438 acc: 0.8608638048171997
step: 1075
train: loss: 1185.7755126953125 acc: 0.8287404179573059  val: loss: 1000.2051391601562 acc: 0.8686047196388245
step: 1080
train: loss: 803.8359375 acc: 0.8704373836517334  val: loss: 807.6805419921875 acc: 0.8639222383499146
step: 1085
train: loss: 455.0972900390625 acc: 0.9348124265670776  val: loss: 1375.14208984375 acc: 0.8362721800804138
step: 1090
train: loss: 819.0202026367188 acc: 0.8673251867294312  val: loss: 1267.1165771484375 acc: 0.8439034223556519
step: 1095
train: loss: 598.73974609375 acc: 0.8904616832733154  val: loss: 1948.0523681640625 acc: 0.7477340698242188
step: 1100
train: loss: 575.5997314453125 acc: 0.9019302725791931  val: loss: 830.4699096679688 acc: 0.8903321027755737
step: 1105
train: loss: 728.716064453125 acc: 

step: 1420
train: loss: 448.4581604003906 acc: 0.9294744729995728  val: loss: 1266.992431640625 acc: 0.7658417224884033
step: 1425
train: loss: 582.9100341796875 acc: 0.8962897062301636  val: loss: 894.8624877929688 acc: 0.8514227867126465
step: 1430
train: loss: 873.2326049804688 acc: 0.7945483326911926  val: loss: 459.2456359863281 acc: 0.914553701877594
step: 1435
train: loss: 665.5408325195312 acc: 0.8886141180992126  val: loss: 859.3973999023438 acc: 0.8380088806152344
step: 1440
train: loss: 1252.9765625 acc: 0.7883152961730957  val: loss: 834.0591430664062 acc: 0.8956588506698608
step: 1445
train: loss: 621.1747436523438 acc: 0.9138297438621521  val: loss: 1365.9991455078125 acc: 0.828635573387146
step: 1450
train: loss: 845.2828979492188 acc: 0.8828544616699219  val: loss: 1590.5091552734375 acc: 0.7664828896522522
step: 1455
train: loss: 498.974853515625 acc: 0.8968296647071838  val: loss: 1541.1724853515625 acc: 0.7289314866065979
step: 1460
train: loss: 474.3211669921875 acc

...

step: 8500
train: loss: 687.704833984375 acc: 0.900650143623352  val: loss: 1409.8101806640625 acc: 0.7983688116073608
step: 8505
train: loss: 408.391845703125 acc: 0.920957088470459  val: loss: 836.69482421875 acc: 0.8565846681594849
step: 8510
train: loss: 776.7756958007812 acc: 0.884085476398468  val: loss: 962.3833618164062 acc: 0.8789849281311035
step: 8515
train: loss: 648.1157836914062 acc: 0.9117481112480164  val: loss: 2023.659423828125 acc: 0.7843455076217651
step: 8520
train: loss: 615.33642578125 acc: 0.7887818217277527  val: loss: 1269.32373046875 acc: 0.802931547164917
step: 8525
train: loss: 988.6300048828125 acc: 0.8734413385391235  val: loss: 1350.7132568359375 acc: 0.775631844997406
step: 8530
train: loss: 895.8613891601562 acc: 0.8690325617790222  val: loss: 823.646728515625 acc: 0.8963270783424377
step: 8535
train: loss: 533.6116333007812 acc: 0.9120465517044067  val: loss: 1319.608642578125 acc: 0.8578731417655945
step: 8540
train: loss: 700.94140625 acc: 0.8860021

step: 8855
train: loss: 429.2545166015625 acc: 0.90177321434021  val: loss: 1856.115478515625 acc: 0.7406032085418701
step: 8860
train: loss: 803.7786254882812 acc: 0.8673467636108398  val: loss: 898.0857543945312 acc: 0.8932377696037292
step: 8865
train: loss: 766.498779296875 acc: 0.8710805177688599  val: loss: 2163.07080078125 acc: 0.7461322546005249
step: 8870
train: loss: 723.1231689453125 acc: 0.8639578819274902  val: loss: 577.04931640625 acc: 0.8906832933425903
step: 8875
train: loss: 566.288330078125 acc: 0.8984544277191162  val: loss: 701.6502075195312 acc: 0.8537136316299438
step: 8880
train: loss: 685.7084350585938 acc: 0.8112854361534119  val: loss: 2677.42626953125 acc: 0.7110626697540283
step: 8885
train: loss: 728.3138427734375 acc: 0.8881998658180237  val: loss: 2170.997802734375 acc: 0.7253420948982239
step: 8890
train: loss: 501.41632080078125 acc: 0.9126753211021423  val: loss: 946.3541870117188 acc: 0.8454703092575073
step: 8895
train: loss: 579.8384399414062 acc: 

step: 9210
train: loss: 469.4143981933594 acc: 0.918169379234314  val: loss: 1157.838623046875 acc: 0.8320972919464111
step: 9215
train: loss: 495.0947570800781 acc: 0.9221024513244629  val: loss: 907.3358764648438 acc: 0.882439911365509
step: 9220
train: loss: 537.4715576171875 acc: 0.9134214520454407  val: loss: 481.8546142578125 acc: 0.920678973197937
step: 9225
train: loss: 797.101318359375 acc: 0.8655365705490112  val: loss: 1590.71630859375 acc: 0.7835911512374878
step: 9230
train: loss: 757.6064453125 acc: 0.8645948767662048  val: loss: 825.3831787109375 acc: 0.8734874129295349
step: 9235
train: loss: 814.8681030273438 acc: 0.8357874155044556  val: loss: 435.66864013671875 acc: 0.902341902256012
step: 9240
train: loss: 697.2196044921875 acc: 0.8836809992790222  val: loss: 950.3919677734375 acc: 0.8678218126296997
step: 9245
train: loss: 358.7632141113281 acc: 0.9432554841041565  val: loss: 704.6840209960938 acc: 0.8843609094619751
step: 9250
train: loss: 778.4578247070312 acc: 0

step: 9565
train: loss: 312.3390808105469 acc: 0.9452136754989624  val: loss: 1562.0858154296875 acc: 0.7845171689987183
step: 9570
train: loss: 693.4005737304688 acc: 0.8647830486297607  val: loss: 889.2564697265625 acc: 0.8392196893692017
step: 9575
train: loss: 971.1431884765625 acc: 0.8573188781738281  val: loss: 1607.0361328125 acc: 0.8423718810081482
step: 9580
train: loss: 613.978759765625 acc: 0.882280707359314  val: loss: 1692.768798828125 acc: 0.7969653606414795
step: 9585
train: loss: 598.9357299804688 acc: 0.8861812949180603  val: loss: 657.6634521484375 acc: 0.8965150117874146
step: 9590
train: loss: 742.8358764648438 acc: 0.8379669785499573  val: loss: 1792.3336181640625 acc: 0.7690132260322571
step: 9595
train: loss: 947.204833984375 acc: 0.8588352203369141  val: loss: 1264.194580078125 acc: 0.7876793146133423
step: 9600
train: loss: 585.4906005859375 acc: 0.8914563059806824  val: loss: 1519.0164794921875 acc: 0.7756802439689636
step: 9605
train: loss: 429.3856201171875 

step: 9920
train: loss: 1076.327392578125 acc: 0.8336434364318848  val: loss: 2047.62841796875 acc: 0.7329540252685547
step: 9925
train: loss: 481.42535400390625 acc: 0.9055980443954468  val: loss: 1691.3907470703125 acc: 0.7253077626228333
step: 9930
train: loss: 798.6771850585938 acc: 0.8409454822540283  val: loss: 1526.6142578125 acc: 0.7664470076560974
step: 9935
train: loss: 519.7805786132812 acc: 0.9067558646202087  val: loss: 884.3716430664062 acc: 0.8576140403747559
step: 9940
train: loss: 839.7199096679688 acc: 0.8386643528938293  val: loss: 1549.58349609375 acc: 0.7832158803939819
step: 9945
train: loss: 856.1129150390625 acc: 0.8671003580093384  val: loss: 1101.0050048828125 acc: 0.8374244570732117
step: 9950
train: loss: 455.7612609863281 acc: 0.9223410487174988  val: loss: 1920.9176025390625 acc: 0.6916379332542419
step: 9955
train: loss: 865.54052734375 acc: 0.8470934629440308  val: loss: 1859.4483642578125 acc: 0.7788092494010925
step: 9960
train: loss: 350.4779968261719